#TRABAJO 1 Recolección de datos con PRAW

Bastian Cortez Arce

Bruno Sainz Silva


In [2]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 3.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import praw
import datetime
from google.colab import userdata
import time
import random

reddit = praw.Reddit(
    client_id=userdata.get('id_client_redit'),
    client_secret=userdata.get('client_secret_redit'),
    user_agent=userdata.get('user_agent_redit'),
    check_for_async=False
)

#funcion para crear respaldo en caso de fallar
def guardar_respaldo(nombre_base):
    df_posts = pd.DataFrame(datosPost)
    df_comentarios = pd.DataFrame(comentariosPost)
    df_posts.to_csv(f"{nombre_base}_posts.csv", index=False)
    df_comentarios.to_csv(f"{nombre_base}_comentarios.csv", index=False)

datosPost = {
    'id_post': [],
    'titulo_post': [],
    "fecha_post": [],
    "autor_post": [],
    "URL": [],
    "numero_comentarios": [],
    "puntaje": [],
    "upvotesRatio": [],
    "texto_post": [],
    "tiene_texto": [],
}

comentariosPost = {
    'id_post': [],
    'id_comentario': [],
    'autor_comentario': [],
    'texto_comentario': [],
    'fecha_comentario': [],
    'puntaje_comentario': [],
    'edited': [],
}

def extraccion_info_post(post):
    fecha = datetime.datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')
    autor = str(post.author) if post.author else "ELIMINADO"
    tiene_texto = "Sí" if post.selftext else "No"

    datosPost['id_post'].append(post.id)
    datosPost['titulo_post'].append(post.title)
    datosPost['fecha_post'].append(fecha)
    datosPost['autor_post'].append(autor)
    datosPost['URL'].append(post.url)
    datosPost['puntaje'].append(post.score)
    datosPost['numero_comentarios'].append(post.num_comments)
    datosPost['upvotesRatio'].append(post.upvote_ratio)
    datosPost['texto_post'].append(post.selftext)
    datosPost['tiene_texto'].append(tiene_texto)

    if post.num_comments > 15:
        post.comments.replace_more(limit=0)
        for comment in post.comments[:10]:
            if not comment.stickied:
                comentariosPost['id_post'].append(post.id)
                comentariosPost['id_comentario'].append(comment.id)
                comentariosPost['autor_comentario'].append(str(comment.author) if comment.author else "ELIMINADO")
                comentariosPost['texto_comentario'].append(comment.body)
                comentariosPost['fecha_comentario'].append(
                    datetime.datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
                )
                comentariosPost['puntaje_comentario'].append(comment.score)
                comentariosPost['edited'].append(comment.edited)

ListaSubreddits = ["Askmen", "Askwomen","askscience"]


#Consultar Reddit
try:
    for subs in ListaSubreddits:
        sub = reddit.subreddit(subs)

        #si subs esta en esta lista, entonces buscamos los posts nuevos
        if subs in ["Askmen", "Askwomen"]:
            submissions = sub.new(limit=200)
        #sino, se consultara el top del año
        elif subs == "askscience":
            submissions = sub.top(limit=200, time_filter='year')

        #se consulta con un tiempo de espera para evitar rechazo de la web
        for i, submission in enumerate(submissions):
            wait = random.randint(4, 8)
            time.sleep(wait)
            print(f"reddit {subs}, post {i+1} --- {wait} [s] ")

            try:
                extraccion_info_post(submission)
                guardar_respaldo(nombre_base=f"askcienceonly_{subs}")  # Se sobrescribe cada vez
            except Exception as e:
                print(f"⚠️ Error al procesar post {i+1} en {subs}: {e}")
                guardar_respaldo(nombre_base="respaldo_error")  # Un solo respaldo general
                raise e

    # Guardado final, tanto la combinacion de post y comentarios como los autoguardados por separado
    df_posts = pd.DataFrame(datosPost)
    df_comentarios = pd.DataFrame(comentariosPost)
    df_pandas = df_posts.merge(df_comentarios, on='id_post', how='inner')
    df_pandas.to_csv('reddit_data.csv', index=False)

except Exception as final_error:
    print(f"🚨 Script detenido por error: {final_error}")


Las librerías con las que trabajaremos serán:


* https://www.reddit.com/r/askscience/

* https://www.reddit.com/r/AskMen/

* https://www.reddit.com/r/AskWomen/

Los subreddits de AskMen y AskWomen fueron elegidos pensando en el futuro, con la intención de poder comparar publicaciones y analizar cómo se comportan según el género, observando las palabras más usadas, el tiempo entre publicaciones, etc. Por esta razón, filtramos los posts más recientes, de modo que contemos con una línea temporal coherente para realizar las comparaciones.



El subreddit de AskScience se utilizará para observar qué tipo de preguntas hace la gente, o bien para darle un uso a los datos entrenando algún modelo enfocado en el área científica. En el mejor de los casos, incluso podría emplearse un modelo ya entrenado para analizar las respuestas a estas dudas: ¿están correctas o son inconclusas?